In [702]:
#profitableHospital-project

In [703]:
#dependencies
import pandas as pd
import requests
import json
import scipy.stats as st
from scipy.stats import linregress
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import tensorflow
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import joblib

In [704]:
# ... File Imports

In [705]:
#2017 Hospital Data from Centers for Medicare Services
df = pd.read_csv("Medicare_Hospital_Cost_Report_PUF_2017.csv")
df.head(2)

C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (105) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,County,Medicare CBSA Number,Rural Versus Urban,CCN Facility Type,...,Total Other Income,Total Income,Total Other Expenses,Net Income,Cost To Charge Ratio,Net Revenue from Medicaid,Medicaid Charges,Net Revenue from Stand-Alone SCHIP,Stand-Alone SCHIP Charges,rpt_rec_num
0,510013,REYNOLDS MEMORIAL HOSPITAL,800 WHEELING AVENUE,GLEN DALE,WV,26038,MARSHALL,48540.0,U,STH,...,4492364.0,2638799.0,NaN,2638799.0,0.453498,1237912.0,4026663.0,NaN,NaN,599181
1,340106,SANDHILLS REGIONAL MEDICAL CENTER,1000 WEST HAMLET AVE,HAMLET,NC,28345,RICHMOND,99934.0,R,STH,...,31174.0,-1114660.0,NaN,-1114660.0,0.173114,391215.0,4981119.0,NaN,NaN,601095


In [706]:
#Quick Check
df['Provider CCN'].nunique()

6075

In [707]:
#Quick Check
df['Provider CCN'].count()

6171

In [708]:
df.drop_duplicates(inplace = True)
df['Provider CCN'].count()

6171

In [709]:
#Certificate of Need Status by State File
conTable = pd.read_csv("certificateofneedstatesTable.csv")
conTable.head(2)

,State Code,CON
0,AK,1
1,AL,1


In [710]:
#String to Numeric Conversion Table for Urban v. Rural designation
urbanTable = pd.read_csv("ruralversusurbanTable.csv")
urbanTable.head()

,Rural Versus Urban,Urban
0,U,1
1,R,0


In [711]:
#Lattitude & Longitude for the 2017 hospital listing
geoTable = pd.read_csv("ccn2geocode.csv")
geoTable.head(2)

,Provider CCN,lat,lng
0,10001,31.215884,-85.363008
1,10005,34.222103,-86.160908


In [712]:
#Quick Check
geoTable['Provider CCN'].count()

6075

In [713]:
# geoTable.drop_duplicates(inplace = True)
# geoTable['Provider CCN'].count()
#df.count()

In [714]:
# ... Appending of source files into one main working file

In [715]:
mainFile1 = df

In [716]:
#Append a lattitude & longitude
mainFile1 = pd.merge(mainFile1, geoTable, how= "left", on=['Provider CCN'])
mainFile1.head(2)

,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,County,Medicare CBSA Number,Rural Versus Urban,CCN Facility Type,...,Total Other Expenses,Net Income,Cost To Charge Ratio,Net Revenue from Medicaid,Medicaid Charges,Net Revenue from Stand-Alone SCHIP,Stand-Alone SCHIP Charges,rpt_rec_num,lat,lng
0,510013,REYNOLDS MEMORIAL HOSPITAL,800 WHEELING AVENUE,GLEN DALE,WV,26038,MARSHALL,48540.0,U,STH,...,NaN,2638799.0,0.453498,1237912.0,4026663.0,NaN,NaN,599181,39.946236,-80.752966
1,340106,SANDHILLS REGIONAL MEDICAL CENTER,1000 WEST HAMLET AVE,HAMLET,NC,28345,RICHMOND,99934.0,R,STH,...,NaN,-1114660.0,0.173114,391215.0,4981119.0,NaN,NaN,601095,34.901612,-79.708561


In [717]:
#Quick Check
mainFile1['Provider CCN'].count()

6171

In [718]:
# mainFile1.drop_duplicates(inplace = True)
# mainFile1['Provider CCN'].count()
#df.count()

In [719]:
#Append certificate of need indicator
mainFile1 = pd.merge(mainFile1, conTable, how= "left", on=['State Code'])
#mainFile1.head()

In [720]:
#Quick Check
mainFile1['Provider CCN'].count()

6171

In [721]:
#Append a numeric representation of rural v. urban
mainFile1 = pd.merge(mainFile1, urbanTable, how= "left", on=['Rural Versus Urban'])

In [722]:
#Quick Check
mainFile1['Provider CCN'].count()

6171

In [723]:
# mainFile1 = mainFile1.dropna(how='any')
# mainFile1['Provider CCN'].count()

In [724]:
mainFile1.head(2)

,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,County,Medicare CBSA Number,Rural Versus Urban,CCN Facility Type,...,Cost To Charge Ratio,Net Revenue from Medicaid,Medicaid Charges,Net Revenue from Stand-Alone SCHIP,Stand-Alone SCHIP Charges,rpt_rec_num,lat,lng,CON,Urban
0,510013,REYNOLDS MEMORIAL HOSPITAL,800 WHEELING AVENUE,GLEN DALE,WV,26038,MARSHALL,48540.0,U,STH,...,0.453498,1237912.0,4026663.0,NaN,NaN,599181,39.946236,-80.752966,1.0,1.0
1,340106,SANDHILLS REGIONAL MEDICAL CENTER,1000 WEST HAMLET AVE,HAMLET,NC,28345,RICHMOND,99934.0,R,STH,...,0.173114,391215.0,4981119.0,NaN,NaN,601095,34.901612,-79.708561,1.0,0.0


In [725]:
mainFile1.to_csv("main1_2017.csv", index=False, header=True)

In [726]:
#mainFile1['Provider CCN'].duplicated()

In [727]:
# mainFile1[mainFile1['Provider CCN'].duplicated(keep = False)].head(200)

In [728]:
# mainFile1['Provider CCN'].drop_duplicates(inplace = True)
# mainFile1['Provider CCN'].count()
#df.count()

In [729]:
# ... Add calculated fields

In [730]:
mainFile2 = mainFile1

In [731]:
#labor Expense
lbrexpense = mainFile2["Wage-Related Costs (Core)"] + mainFile2["Total Salaries (adjusted)"]
mainFile2["Labor Exp"] = lbrexpense
#wagepercent.head()

In [732]:
#Append calculated field inpatient percent
inpatient = mainFile2["Inpatient Revenue"]/mainFile2["Gross Revenue"]
mainFile2["Inpatient %"] = inpatient
#mainFile2.head()

In [733]:
#Append calculated field outpatient percent
outpatient = mainFile2["Outpatient Revenue"]/mainFile2["Gross Revenue"]
mainFile2["Outpatient %"] = outpatient
#mainFile2.head()

In [734]:
#Append calculated field charity percent
charity = mainFile2["Cost of Charity Care"]/mainFile2["Net Patient Revenue"]
mainFile2["Charity %"] = charity
#mainFile2.head()

In [735]:
#Append calculated field uncompensated percent
uncompensated = mainFile2["Cost of Uncompensated Care"]/mainFile2["Net Patient Revenue"]
mainFile2["Uncompensated %"] = charity
#mainFile2.head()

In [736]:
#Append calculated field revenue per discharge
revdischarge = mainFile2["Net Patient Revenue"]/mainFile2["Total Discharges (V + XVIII + XIX + Unknown)"]
mainFile2["Rev/Discharge"] = revdischarge
#mainFile2.head()

In [737]:
#Append calculated field revenue per bed
revbed = mainFile2["Net Patient Revenue"]/mainFile2["Number of Beds"]
mainFile2["Rev Per Bed"] = revbed
#mainFile2.head()

In [738]:
#Append calculated field revenue per bed day
revbed = mainFile2["Net Patient Revenue"]/mainFile2["Total Bed Days Available"]
mainFile2["Rev Per Bed Days"] = revbed
#mainFile2.head()

In [739]:
#Append calculated field revs per fte
revfte = mainFile2["Net Patient Revenue"]/mainFile2["FTE - Employees on Payroll"]
mainFile2["Rev Per FTE"] = revfte
#mainFile2.head()

In [740]:
#Append calculated field discharges per bed
volbed = mainFile2["Total Discharges (V + XVIII + XIX + Unknown)"]/mainFile2["Number of Beds"]
mainFile2["Discharges Per Bed"] = volbed
#mainFile2.head()

In [741]:
#Append calculated field discharges per bed day
voldays = mainFile2["Total Discharges (V + XVIII + XIX + Unknown)"]/mainFile2["Total Bed Days Available"]
mainFile2["Discharges Per Bed Days"] = voldays
#mainFile2.head()

In [742]:
#Append calculated field discharges per bed
volfte = mainFile2["Total Discharges (V + XVIII + XIX + Unknown)"]/mainFile2["FTE - Employees on Payroll"]
mainFile2["Discharge/FTE"] = volbed
#mainFile2.head()

In [743]:
#labor percent
wagepercent = mainFile2["Wage-Related Costs (Core)"] + mainFile2["Total Salaries (adjusted)"]
wagepercent = wagepercent / mainFile2["Net Patient Revenue"]
mainFile2["Labor %"] = wagepercent
#wagepercent.head()

In [744]:
#Overhead percent
overheadpercent = mainFile2["Overhead Non-Salary Costs"] / mainFile2["Net Patient Revenue"]
mainFile2["Overhead %"] = overheadpercent
#overheadpercent.head()

In [745]:
#Depreciation percent
deprpercent = mainFile2["Depreciation Cost"] / mainFile2["Net Patient Revenue"]
mainFile2["Depreciation %"] = deprpercent
#overheadpercent.head()

In [746]:
#labor Per FTE
wagefte = mainFile2["Wage-Related Costs (Core)"] + mainFile2["Total Salaries (adjusted)"]
wagefte = wagefte / mainFile2["FTE - Employees on Payroll"]
mainFile2["LbrExp/FTE"] = wagefte
#wagepercent.head()

In [747]:
#Overhead Expense Per Bed
overbed = mainFile2["Overhead Non-Salary Costs"] / mainFile2["Number of Beds"]
mainFile2["Overhead Exp Per Bed"] = overbed
#wagepercent.head()

In [748]:
#Append calculated field net income percent to file
profit = mainFile2["Net Income"]/mainFile2["Net Patient Revenue"]
mainFile2["Net Income %"] = profit
#mainFile2.head()

In [749]:
# Review net income percent for outlier reduction approach
netinc = mainFile2["Net Income %"]
#netinc.head()

In [750]:
# Box plot and outlier calculations for outlier approach
netinc2 = netinc.dropna(how='any')
chartdata = netinc2
# fig1, ax1 = plt.subplots()
# ax1.set_title('Net Income Percent')
# ax1.set_ylabel('Net Income Percent')
# ax1.boxplot(chartdata)
# plt.show()

In [751]:
#Quartile calculations
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile of profit percent is: {lowerq}")
print(f"The upper quartile of profit percent is: {upperq}")
print(f"The interquartile range of profit percent is: {iqr}")
print(f"The the median of profit percent is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile of profit percent is: -0.030653238847723918
The upper quartile of profit percent is: 0.11379730445563284
The interquartile range of profit percent is: 0.14445054330335677
The the median of profit percent is: 0.0380451602645128 
Values below -0.24732905380275907 could be outliers.
Values above 0.330473119410668 could be outliers.


In [752]:
#Create a bronze, silver, gold score for clasifying net income performance
bins = [-100, -0.030605, 0.113797, 100]
group_names = ["3", "2", "1"]

In [753]:
#Add a bronze, silver, gold scoring system to the dataset to support machine learning
mainFile2["Net Income Score"] = pd.cut(mainFile2["Net Income %"], bins, labels=group_names, include_lowest=True)
#mainFile2.head()

In [754]:
#Quick check on distribution of bronze, silver and gold
mainFile2['Net Income Score'].value_counts()

2    2945
1    1473
3    1473
Name: Net Income Score, dtype: int64

In [755]:
#Pacesetter status
bins2 = [-100, -0.030605, .038045,  0.11379, 100]
group_names2 = ["4th Quartile", "3rd Quartile", "2nd Quartile", "1st Quartile"]

In [756]:
#Add Pacesetter Status to dataframe
mainFile2["Pacesetter Group"] = pd.cut(mainFile2["Net Income %"], bins2, labels=group_names2, include_lowest=True)
#mainFile2.head()

In [757]:
mainFile2['Pacesetter Group'].value_counts()

1st Quartile    1473
2nd Quartile    1473
4th Quartile    1473
3rd Quartile    1472
Name: Pacesetter Group, dtype: int64

In [758]:
#Quick Check
mainFile2['Provider CCN'].count()

6171

In [759]:
#mainFile2.to_csv("main2_2017.csv", index=False, header=True)

In [760]:
# ..... File targeting

In [761]:
mainFile3 = mainFile2
mainFile3['Provider CCN'].count()

6171

In [762]:
mainFile3['lat'].dtype

dtype('float64')

In [763]:
#Remove the Sitka exceptions
mainFile3 = mainFile3.loc[mainFile3['lat'] <= 48.85]

In [764]:
#Remove the European exceptions
#mainFile3 = mainFile3.loc[mainFile3['lat'] <= -60]

In [765]:
#Targets dataset on Colorad0
#mainFile3 = mainFile3.loc[df['State Code'] == 'CO']

In [766]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'PR']

In [767]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'GU']

In [768]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'VI']

In [769]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'MP']

In [770]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'HI']

In [771]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'AK']

In [772]:
#Quick Check ... Lose 117 to state code reduction
mainFile3['Provider CCN'].count()

6026

In [773]:
#Remove hospitals that did not report revenue (Lose 273 to missing or unreported revenue events)
mainFile3 = mainFile3.loc[mainFile3['Net Patient Revenue'] >= 0.1]

In [774]:
#Quick Check
mainFile3['Provider CCN'].count()

5756

In [775]:
#Remove hospitals that discharged less than 15 patients (Lose 49 by placing a 15 patient minimum for study eligibility)
mainFile3 = mainFile3.loc[mainFile3['Total Discharges (V + XVIII + XIX + Unknown)'] >= 15]

In [776]:
#Quick Check
mainFile3['Provider CCN'].count()

5708

In [777]:
#Remove hospitals with less than 10 employees on payroll (Lose 23 by placing a 10 employee minimum for study eligibility)
mainFile3 = mainFile3.loc[mainFile3['FTE - Employees on Payroll'] >= 10]

In [778]:
#Quick Check
mainFile3['Provider CCN'].count()

5685

In [779]:
#Targets dataset on certificate of need states
#mainFile3 = mainFile3.loc[mainFile2['CON'] == 1]

In [780]:
#Remove net income outliers
#mainFile3 = mainFile3.loc[mainFile2['Net Income %'] <= .3304]

In [781]:
#Remove net income outliers
#mainFile3 = mainFile3.loc[mainFile2['Net Income %'] >= -.2473]

In [782]:
#Remove Discharges Per Bed outliers
#mainFile3 = mainFile3.loc[mainFile2['Rev Per Discharge'] <= 90000]

In [783]:
#Remove Rev Per Bed outliers
#mainFile3 = mainFile3.loc[mainFile2['Rev Per Bed'] <= 2000000]

In [784]:
#Remove Rev Per FTE outliers
#mainFile3 = mainFile3.loc[mainFile2['Rev Per FTE'] <= 500000]

In [785]:
#Remove Rev Per Discharge outliers
#mainFile3 = mainFile3.loc[mainFile2['Rev Per Discharge'] <= 175000]

In [786]:
#Targets dataset on STH locations (~56% of eligible continental US locations are STH providers)
mainFile3 = mainFile3.loc[mainFile3['CCN Facility Type'] == 'STH']

In [787]:
#Quick Check
mainFile3['Provider CCN'].count()

3184

In [788]:
#Quick check on distribution of bronze, silver and gold
#mainFile3['Net Income Score'].value_counts()

In [789]:
#Quick Check
mainFile3['Provider CCN'].count()

3184

In [790]:
#mainFile3.to_csv("main3_2017.csv", index=False, header=True)

In [791]:
# ... Target file based on relevant features

In [792]:
mainFile4 = mainFile3

In [793]:
# #%'s'
# chg1percent = mainFile4["Net Income %"]
# chg1percent = chg1percent *100
# mainFile4["Net Income %"] = chg1percent
# mainFile4["Net Income %"] = mainFile4["Net Income %"].map("{:.4f}%".format)

# chg2percent = mainFile4["Labor %"]
# chg2percent = chg2percent *100
# mainFile4["Labor %"] = chg2percent
# mainFile4["Labor %"] = mainFile4["Labor %"].map("{:.4f}%".format)

# chg3percent = mainFile4["Overhead %"]
# chg3percent = chg3percent *100
# mainFile4["Overhead %"] = chg3percent
# mainFile4["Overhead %"] = mainFile4["Overhead %"].map("{:.4f}%".format)

# chg4percent = mainFile4["Depreciation %"]
# chg4percent = chg4percent *100
# mainFile4["Depreciation %"] = chg4percent
# mainFile4["Depreciation %"] = mainFile4["Depreciation %"].map("{:.4f}%".format)

# chg5percent = mainFile4["Charity %"]
# chg5percent = chg5percent *100
# mainFile4["Charity %"] = chg5percent
# mainFile4["Charity %"] = mainFile4["Charity %"].map("{:.4f}%".format)

# chg6percent = mainFile4["Uncompensated %"]
# chg6percent = chg6percent *100
# mainFile4["Uncompensated %"] = chg6percent
# mainFile4["Uncompensated %"] = mainFile4["Uncompensated %"].map("{:.4f}%".format)

# chg7percent = mainFile4["Inpatient %"]
# chg7percent = chg7percent *100
# mainFile4["Inpatient %"] = chg7percent
# mainFile4["Inpatient %"] = mainFile4["Inpatient %"].map("{:.4f}%".format)

# chg8percent = mainFile4["Outpatient %"]
# chg8percent = chg8percent *100
# mainFile4["Outpatient %"] = chg8percent
# mainFile4["Outpatient %"] = mainFile4["Outpatient %"].map("{:.4f}%".format)

# #$'s'
# mainFile4["Net Patient Revenue"] = mainFile4["Net Patient Revenue"].map("${:,.0f}".format)
# mainFile4["Net Income"] = mainFile4["Net Income"].map("${:.0f}".format)
# mainFile4["Wage-Related Costs (Core)"] = mainFile4["Wage-Related Costs (Core)"].map("${:,.0f}".format)
# mainFile4["Total Salaries (adjusted)"] = mainFile4["Total Salaries (adjusted)"].map("${:,.0f}".format)
# mainFile4["Contract Labor"] = mainFile4["Contract Labor"].map("${:,.0f}".format)
# mainFile4["Rev Per FTE"] = mainFile4["Rev Per FTE"].map("${:,.0f}".format)
# mainFile4["Rev/Discharge"] = mainFile4["Rev/Discharge"].map("${:,.0f}".format)
# mainFile4["Rev Per Bed"] = mainFile4["Rev Per Bed"].map("${:,.0f}".format)
# mainFile4["Rev Per Bed Days"] = mainFile4["Rev Per Bed Days"].map("${:,.0f}".format)
# mainFile4["LbrExp/FTE"] = mainFile4["LbrExp/FTE"].map("${:,.0f}".format)
# mainFile4["Overhead Exp Per Bed"] = mainFile4["Overhead Exp Per Bed"].map("${:,.0f}".format)

# ##'s'
# mainFile4["Discharges Per Bed"] = mainFile4["Discharges Per Bed"].map("{:,.2f}".format)
# mainFile4["Discharges Per Bed Days"] = mainFile4["Discharges Per Bed Days"].map("{:,.2f}".format)
# mainFile4["Discharge/FTE"] = mainFile4["Discharge/FTE"].map("{:,.2f}".format)

In [794]:
#Targeting the dataset on metrics important to Kent
mainFile4 = mainFile4[['Provider CCN','Hospital Name','Street Address','City','State Code','Zip Code', 'lat', 'lng', 'Rural Versus Urban', 'CCN Facility Type', 'Fiscal Year Begin Date', 'Fiscal Year End Date', 'FTE - Employees on Payroll', 'Number of Beds', 'Total Bed Days Available', 'Total Discharges (V + XVIII + XIX + Unknown)', 'Net Patient Revenue', 'Net Income', 'Wage-Related Costs (Core)', 'Total Salaries (adjusted)', 'Contract Labor', 'Rev Per FTE', 'CON','Urban','Inpatient %','Outpatient %','Charity %','Uncompensated %','Rev/Discharge','Rev Per Bed','Rev Per Bed Days', 'Discharges Per Bed','Discharges Per Bed Days','Discharge/FTE','Net Income %', 'Net Income Score', 'Labor %', 'Overhead %', 'Depreciation %', 'LbrExp/FTE', 'Overhead Exp Per Bed', 'Pacesetter Group']]
mainFile4.head(2)


,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,lat,lng,Rural Versus Urban,CCN Facility Type,...,Discharges Per Bed Days,Discharge/FTE,Net Income %,Net Income Score,Labor %,Overhead %,Depreciation %,LbrExp/FTE,Overhead Exp Per Bed,Pacesetter Group
0,510013,REYNOLDS MEMORIAL HOSPITAL,800 WHEELING AVENUE,GLEN DALE,WV,26038,39.946236,-80.752966,U,STH,...,0.067547,6.214286,0.308395,1,0.715116,0.627406,0.031451,18996.972990,76691.942857,1st Quartile
1,340106,SANDHILLS REGIONAL MEDICAL CENTER,1000 WEST HAMLET AVE,HAMLET,NC,28345,34.901612,-79.708561,R,STH,...,0.050205,3.062500,-0.337043,3,0.509928,0.924882,0.073496,61909.838473,47792.984375,4th Quartile


In [795]:
# Identify incomplete rows
mainFile4.count()

Provider CCN                                    3184
Hospital Name                                   3184
Street Address                                  3184
City                                            3184
State Code                                      3184
Zip Code                                        3184
lat                                             3184
lng                                             3184
Rural Versus Urban                              3184
CCN Facility Type                               3184
Fiscal Year Begin Date                          3184
Fiscal Year End Date                            3184
FTE - Employees on Payroll                      3184
Number of Beds                                  3184
Total Bed Days Available                        3184
Total Discharges (V + XVIII + XIX + Unknown)    3184
Net Patient Revenue                             3184
Net Income                                      3183
Wage-Related Costs (Core)                     

In [796]:
mainFile4 = mainFile4.dropna(how='any')

In [797]:
#mainFile4 = mainFile4.dropna(how='nan')

In [798]:
# Identify incomplete rows
mainFile4.count()

Provider CCN                                    2705
Hospital Name                                   2705
Street Address                                  2705
City                                            2705
State Code                                      2705
Zip Code                                        2705
lat                                             2705
lng                                             2705
Rural Versus Urban                              2705
CCN Facility Type                               2705
Fiscal Year Begin Date                          2705
Fiscal Year End Date                            2705
FTE - Employees on Payroll                      2705
Number of Beds                                  2705
Total Bed Days Available                        2705
Total Discharges (V + XVIII + XIX + Unknown)    2705
Net Patient Revenue                             2705
Net Income                                      2705
Wage-Related Costs (Core)                     

In [799]:
# #%'s'
# chg1percent = mainFile4["Net Income %"]
# chg1percent = chg1percent *100
# mainFile4["Net Income %"] = chg1percent
# mainFile4["Net Income %"] = mainFile4["Net Income %"].map("{:.4f}%".format)

# chg2percent = mainFile4["Labor %"]
# chg2percent = chg2percent *100
# mainFile4["Labor %"] = chg2percent
# mainFile4["Labor %"] = mainFile4["Labor %"].map("{:.4f}%".format)

# chg3percent = mainFile4["Overhead %"]
# chg3percent = chg3percent *100
# mainFile4["Overhead %"] = chg3percent
# mainFile4["Overhead %"] = mainFile4["Overhead %"].map("{:.4f}%".format)

# chg4percent = mainFile4["Depreciation %"]
# chg4percent = chg4percent *100
# mainFile4["Depreciation %"] = chg4percent
# mainFile4["Depreciation %"] = mainFile4["Depreciation %"].map("{:.4f}%".format)

# chg5percent = mainFile4["Charity %"]
# chg5percent = chg5percent *100
# mainFile4["Charity %"] = chg5percent
# mainFile4["Charity %"] = mainFile4["Charity %"].map("{:.4f}%".format)

# chg6percent = mainFile4["Uncompensated %"]
# chg6percent = chg6percent *100
# mainFile4["Uncompensated %"] = chg6percent
# mainFile4["Uncompensated %"] = mainFile4["Uncompensated %"].map("{:.4f}%".format)

# chg7percent = mainFile4["Inpatient %"]
# chg7percent = chg7percent *100
# mainFile4["Inpatient %"] = chg7percent
# mainFile4["Inpatient %"] = mainFile4["Inpatient %"].map("{:.4f}%".format)

# chg8percent = mainFile4["Outpatient %"]
# chg8percent = chg8percent *100
# mainFile4["Outpatient %"] = chg8percent
# mainFile4["Outpatient %"] = mainFile4["Outpatient %"].map("{:.4f}%".format)

# #$'s'
# mainFile4["Net Patient Revenue"] = mainFile4["Net Patient Revenue"].map("${:,.0f}".format)
# mainFile4["Net Income"] = mainFile4["Net Income"].map("${:.0f}".format)
# mainFile4["Wage-Related Costs (Core)"] = mainFile4["Wage-Related Costs (Core)"].map("${:,.0f}".format)
# mainFile4["Total Salaries (adjusted)"] = mainFile4["Total Salaries (adjusted)"].map("${:,.0f}".format)
# mainFile4["Contract Labor"] = mainFile4["Contract Labor"].map("${:,.0f}".format)
# mainFile4["Rev Per FTE"] = mainFile4["Rev Per FTE"].map("${:,.0f}".format)
# mainFile4["Rev/Discharge"] = mainFile4["Rev/Discharge"].map("${:,.0f}".format)
# mainFile4["Rev Per Bed"] = mainFile4["Rev Per Bed"].map("${:,.0f}".format)
# mainFile4["Rev Per Bed Days"] = mainFile4["Rev Per Bed Days"].map("${:,.0f}".format)
# mainFile4["LbrExp/FTE"] = mainFile4["LbrExp/FTE"].map("${:,.0f}".format)
# mainFile4["Overhead Exp Per Bed"] = mainFile4["Overhead Exp Per Bed"].map("${:,.0f}".format)

# ##'s'
# mainFile4["Discharges Per Bed"] = mainFile4["Discharges Per Bed"].map("{:,.2f}".format)
# mainFile4["Discharges Per Bed Days"] = mainFile4["Discharges Per Bed Days"].map("{:,.2f}".format)
# mainFile4["Discharge/FTE"] = mainFile4["Discharge/FTE"].map("{:,.2f}".format)

In [800]:
#Output targeted file
mainFile4.to_csv("main4_2017.csv", index=False, header=True)

In [801]:
# ... Net Income to feature review

In [802]:
targetFile5 = mainFile4

In [803]:
#Outlier Review
# chartdata = targetFile5["Discharges Per Bed"]
# quartiles = chartdata.quantile([.25,.5,.75])
# lowerq = quartiles[0.25]
# upperq = quartiles[0.75]
# iqr = upperq-lowerq

# print(f"The lower quartile of profit percent is: {lowerq}")
# print(f"The upper quartile of profit percent is: {upperq}")
# print(f"The interquartile range of profit percent is: {iqr}")
# print(f"The the median of profit percent is: {quartiles[0.5]} ")

# lower_bound = lowerq - (1.5*iqr)
# upper_bound = upperq + (1.5*iqr)
# print(f"Values below {lower_bound} could be outliers.")
# print(f"Values above {upper_bound} could be outliers.")

In [804]:
#NI v. discharges per bed
# x_axis = targetFile5["Discharges Per Bed"]
# y_axis = targetFile5["Net Income %"]
# plt.title("Net Income % v. Discharges Per Bed")
# plt.xlabel("Discharges Per Bed")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [805]:
#NI v. discharges per bed days
# x_axis = targetFile5["Discharges Per Bed Days"]
# y_axis = targetFile5["Net Income %"]
# plt.title("Net Income % v. Discharges Per Bed Days")
# plt.xlabel("Discharges Per Bed Days")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# #ax.set_yscale('log')
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [806]:
#Outlier Review
# chartdata = targetFile5["Rev Per Bed"]
# quartiles = chartdata.quantile([.25,.5,.75])
# lowerq = quartiles[0.25]
# upperq = quartiles[0.75]
# iqr = upperq-lowerq

# print(f"The lower quartile of profit percent is: {lowerq}")
# print(f"The upper quartile of profit percent is: {upperq}")
# print(f"The interquartile range of profit percent is: {iqr}")
# print(f"The the median of profit percent is: {quartiles[0.5]} ")

# lower_bound = lowerq - (1.5*iqr)
# upper_bound = upperq + (1.5*iqr)
# print(f"Values below {lower_bound} could be outliers.")
# print(f"Values above {upper_bound} could be outliers.")

In [807]:
#NI v. Revenue Per Bed
# x_axis = targetFile5["Rev Per Bed"]
# y_axis = targetFile5["Net Income %"]
# plt.title("Net Income % v. Rev Per Bed")
# plt.xlabel("Rev Per Bed")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [808]:
#Outlier Review
# chartdata = targetFile5["Rev Per FTE"]
# quartiles = chartdata.quantile([.25,.5,.75])
# lowerq = quartiles[0.25]
# upperq = quartiles[0.75]
# iqr = upperq-lowerq

# print(f"The lower quartile is: {lowerq}")
# print(f"The upper quartile is: {upperq}")
# print(f"The interquartile range is: {iqr}")
# print(f"The the median is: {quartiles[0.5]} ")

# lower_bound = lowerq - (1.5*iqr)
# upper_bound = upperq + (1.5*iqr)
# print(f"Values below {lower_bound} could be outliers.")
# print(f"Values above {upper_bound} could be outliers.")

In [809]:
#NI v. Rev Per FTE
targetFile6 = targetFile5
# targetFile6 = targetFile5.loc[targetFile5['Rev Per FTE'] <= 1000000]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] <= .35]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] >= -.25]
# x_axis = targetFile6["Rev Per FTE"]
# y_axis = targetFile6["Net Income %"]
# plt.title("Net Income % v. Rev Per FTE")
# plt.xlabel("Rev Per FTE")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [810]:
#Outlier Review
# chartdata = targetFile5["Rev/Discharge"]
# quartiles = chartdata.quantile([.25,.5,.75])
# lowerq = quartiles[0.25]
# upperq = quartiles[0.75]
# iqr = upperq-lowerq

# print(f"The lower quartile of profit percent is: {lowerq}")
# print(f"The upper quartile of profit percent is: {upperq}")
# print(f"The interquartile range of profit percent is: {iqr}")
# print(f"The the median of profit percent is: {quartiles[0.5]} ")

# lower_bound = lowerq - (1.5*iqr)
# upper_bound = upperq + (1.5*iqr)
# print(f"Values below {lower_bound} could be outliers.")
# print(f"Values above {upper_bound} could be outliers.")

In [811]:
#NI v. Rev Per Discharge
# x_axis = targetFile5["Rev Per Discharge"]
# y_axis = targetFile5["Net Income %"]
# plt.title("Net Income % v. Rev Per Discharge")
# plt.xlabel("Rev Per Discharge")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [812]:
#Outlier Review
# chartdata = targetFile5["LbrExp/FTE"]
# quartiles = chartdata.quantile([.25,.5,.75])
# lowerq = quartiles[0.25]
# upperq = quartiles[0.75]
# iqr = upperq-lowerq

# print(f"The lower quartile is: {lowerq}")
# print(f"The upper quartile is: {upperq}")
# print(f"The interquartile range is: {iqr}")
# print(f"The the median of is: {quartiles[0.5]} ")

# lower_bound = lowerq - (1.5*iqr)
# upper_bound = upperq + (1.5*iqr)
# print(f"Values below {lower_bound} could be outliers.")
# print(f"Values above {upper_bound} could be outliers.")

In [813]:
#NI v. Labor Expense Per FTE
# targetFile6 = targetFile5.loc[targetFile5['Labor Exp Per FTE'] <= 130000]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] <= .35]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] >= -.25]
# x_axis = targetFile6["Labor Exp Per FTE"]
# y_axis = targetFile6["Net Income %"]
# plt.title("Net Income % v. Labor Exp Per FTE")
# plt.xlabel("Labor Exp Per FTE")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [814]:
#Outlier Review
# chartdata = targetFile5["Overhead %"]
# quartiles = chartdata.quantile([.25,.5,.75])
# lowerq = quartiles[0.25]
# upperq = quartiles[0.75]
# iqr = upperq-lowerq

# print(f"The lower quartile is: {lowerq}")
# print(f"The upper quartile is: {upperq}")
# print(f"The interquartile range is: {iqr}")
# print(f"The the median of is: {quartiles[0.5]} ")

# lower_bound = lowerq - (1.5*iqr)
# upper_bound = upperq + (1.5*iqr)
# print(f"Values below {lower_bound} could be outliers.")
# print(f"Values above {upper_bound} could be outliers.")

In [815]:
#NI v. Overhead Percent
# targetFile6 = targetFile5.loc[targetFile5['Overhead %'] <= .90]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] <= .35]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] >= -.25]
# x_axis = targetFile6["Overhead %"]
# y_axis = targetFile6["Net Income %"]
# plt.title("Net Income % v. Overhead %")
# plt.xlabel("Overhead %")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [816]:
#Outlier Review
# chartdata = targetFile5["Labor %"]
# quartiles = chartdata.quantile([.25,.5,.75])
# lowerq = quartiles[0.25]
# upperq = quartiles[0.75]
# iqr = upperq-lowerq

# print(f"The lower quartile is: {lowerq}")
# print(f"The upper quartile is: {upperq}")
# print(f"The interquartile range is: {iqr}")
# print(f"The the median of is: {quartiles[0.5]} ")

# lower_bound = lowerq - (1.5*iqr)
# upper_bound = upperq + (1.5*iqr)
# print(f"Values below {lower_bound} could be outliers.")
# print(f"Values above {upper_bound} could be outliers.")

In [817]:
#NI v. Labor Percent
targetFile6 = targetFile5
# targetFile6 = targetFile5.loc[targetFile5['Labor %'] <= .90]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] <= .35]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] >= -.25]
# x_axis = targetFile6["Labor %"]
# y_axis = targetFile6["Net Income %"]
# plt.title("Net Income % v. Labor %")
# plt.xlabel("Labor %")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [818]:
#Outlier Review
# chartdata = targetFile5["Discharge/FTE"]
# quartiles = chartdata.quantile([.25,.5,.75])
# lowerq = quartiles[0.25]
# upperq = quartiles[0.75]
# iqr = upperq-lowerq

# print(f"The lower quartile is: {lowerq}")
# print(f"The upper quartile is: {upperq}")
# print(f"The interquartile range is: {iqr}")
# print(f"The the median of is: {quartiles[0.5]} ")

# lower_bound = lowerq - (1.5*iqr)
# upper_bound = upperq + (1.5*iqr)
# print(f"Values below {lower_bound} could be outliers.")
# print(f"Values above {upper_bound} could be outliers.")

In [819]:
#NI v. Labor Percent
# targetFile6 = targetFile5
# targetFile6 = targetFile5.loc[targetFile5['Discharges Per FTE'] <= 120]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] <= .35]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] >= -.25]
# x_axis = targetFile6["Discharges Per FTE"]
# y_axis = targetFile6["Net Income %"]
# plt.title("Net Income % v. Discharges Per FTE")
# plt.xlabel("Discharges Per FTE")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [820]:
#Start machine learning ...

In [821]:
# Select features to be used as x values
features1 = mainFile4[['CON','Urban','Inpatient %','Outpatient %','Charity %','Uncompensated %','Rev/Discharge','Rev Per Bed','Rev Per Bed Days', 'Discharges Per Bed','Discharges Per Bed Days','Discharge/FTE', 'Labor %', 'Overhead %', 'Depreciation %', 'LbrExp/FTE', 'Overhead Exp Per Bed']]
features1.head(2)
#RFE logrithmic output ... array([15, 17, 16, 14, 12,  8, 11,  6,  5 (R/BDay),  7,  9,  3(D/FTE),  2(L%),  1 (OH%),  4 (D%), 13, 10])

,CON,Urban,Inpatient %,Outpatient %,Charity %,Uncompensated %,Rev/Discharge,Rev Per Bed,Rev Per Bed Days,Discharges Per Bed,Discharges Per Bed Days,Discharge/FTE,Labor %,Overhead %,Depreciation %,LbrExp/FTE,Overhead Exp Per Bed
1,1.0,0.0,0.325730,0.674270,0.001075,0.001075,16873.362245,51674.671875,847.125768,3.062500,0.050205,3.062500,0.509928,0.924882,0.073496,61909.838473,47792.984375
5,1.0,0.0,0.364763,0.635237,0.000905,0.000905,15444.529412,132660.378947,1105.503158,8.589474,0.071579,8.589474,0.414191,0.826683,0.119266,21706.316534,109668.063158


In [822]:
#Output features file as a reference document
features1.to_csv("features2017.csv", index=False, header=True)

In [823]:
# ... Linear Regression Model

In [824]:
# Identify the outcome column/data
y = mainFile4[['Net Income %']]
y.head()

,Net Income %
1,-0.337043
5,-0.132426
6,-0.291462
7,-0.050419
8,0.080314


In [825]:
mainFile4["Net Income %"].dtype

dtype('float64')

In [826]:
y.shape

(2705, 1)

In [827]:
X = features1
X.shape

(2705, 17)

In [828]:
#Create a training and testing dataset - one hot
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [829]:
y_train.shape

(2028, 1)

In [830]:
X_train.shape

(2028, 17)

In [831]:
y_test.shape

(677, 1)

In [832]:
X_test.shape

(677, 17)

In [833]:
# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [834]:
#Create the linear regression model and fit to the training data
#score is r2 root mean ... 1 is better
modela = LinearRegression()
modela.fit(X_train_scaled, y_train)
score = modela.score(X_train_scaled, y_train)
print(f"Training Data Score: {modela.score(X_train_scaled, y_train)}")

Training Data Score: 0.4494680637170162


In [835]:
#Run the model with the testing data and determine score
print(f"Testing Data Score: {modela.score(X_test_scaled, y_test)}")

Testing Data Score: -0.29121784281244834


In [836]:
# ... Linear Recursive Feature Elimination

In [837]:
#Identify features to create a more efficient model with
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=5, step=1)
selector = selector.fit(X_train_scaled, y_train)

In [838]:
selector.support_

array([False, False, False, False, False, False, False,  True,  True,
       False,  True, False, False,  True, False, False,  True])

In [839]:
selector.ranking_

array([ 9, 11, 13, 12,  6,  7,  5,  1,  1,  2,  1,  4,  3,  1,  8, 10,  1])

In [840]:
# ... Logistics Model

In [841]:
# Identify the outcome column/data
yy = mainFile4[['Net Income Score']]
yy.head()

,Net Income Score
1,3
5,3
6,3
7,3
8,2


In [842]:
yy.shape

(2705, 1)

In [843]:
XX = features1
XX.shape

(2705, 17)

In [844]:
#Create a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(
    XX, yy, random_state=1)

In [845]:
y_train.shape

(2028, 1)

In [846]:
X_train.shape

(2028, 17)

In [847]:
y_test.shape

(677, 1)

In [848]:
y_test.shape

(677, 1)

In [849]:
# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [850]:
#Create the logistic regression model and fit to the training data
#score is r2 root mean ... 1 is better
modela = LogisticRegression()
modela.fit(X_train_scaled, y_train)
score = modela.score(X_train_scaled, y_train)
print(f"Training Data Score: {modela.score(X_train_scaled, y_train)}")

Training Data Score: 0.5981262327416174


C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [851]:
#Run the model with the testing data and determine score
print(f"Testing Data Score: {modela.score(X_test_scaled, y_test)}")

Testing Data Score: 0.6410635155096012


In [852]:
# ... recursive feature elimination

In [853]:
#Identify features to create a more efficient model with
estimator = LogisticRegression()
selector = RFE(estimator, n_features_to_select=1, step=1)
selector = selector.fit(X_train_scaled, y_train)

C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Pleas

In [854]:
selector.support_

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False])

In [855]:
selector.ranking_

array([14, 17, 15, 13, 11,  9,  7,  6,  5,  8, 12,  3,  2,  1,  4, 16, 10])

In [856]:
#End ... kent matthew rodgers, profitableHospital-project